In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from glob import glob
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, GlobalAveragePooling2D, Dense, Dropout, Flatten, Activation, BatchNormalization
import cv2
import pickle
from scipy.ndimage import median_filter, convolve
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from google.colab import files
sys.version

'3.6.9 (default, Jul 17 2020, 12:50:27) \n[GCC 8.4.0]'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
data_dir = "/content/drive/My Drive/flowers"

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [4]:
data = []
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224,224))
                
                image = np.array(image, dtype=np.float32)
                
                data.append([image, label])
            except Exception as e:
                pass
    print(len(data))
    
    pik = open('data.pickle', 'wb')
    pickle.dump(data,pik)
    pik.close()

In [5]:
make_data()

4346


In [6]:
# zmienić labelki i sprawdzić kod bo array jest niepotrzebny 
def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()
    
    np.random.shuffle(data)
    
    feature = []
    labels = []
    
    for img, label in data:
        feature.append(img)
        labels.append(label)
        
    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    
    feature = feature/255.0
    
    return [feature, labels]

In [7]:
(feature, labels) = load_data()

In [8]:
labels = labels.reshape((-1,1))

In [9]:
x_train, x_test, y_train, y_test = train_test_split (feature, labels, test_size = 0.2, random_state = 0)

In [10]:
x_test.shape

(870, 224, 224, 3)

In [11]:
y_test.shape

(870, 1)

In [15]:
input_shape = (224,224,3)
input = tf.keras.layers.Input(input_shape, dtype = tf.uint8)
x = tf.cast(input, tf.float32)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.5)(x)
x = tf.keras.layers.experimental.preprocessing.RandomFlip()(x)

base_model = keras.applications.Xception(
    weights='imagenet',  
    input_shape=input_shape,
    include_top=False)
base_model.trainable = False
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x) # wyłącza cześć neuronów żeby model się nie przeuczał
output = keras.layers.Dense(5)(x)

model = tf.keras.Model(input, output)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_Cast_1 (TensorFl [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv_1 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub_1 (TensorFlo [(None, 224, 224, 3)]     0         
_________________________________________________________________
random_rotation_1 (RandomRot (None, 224, 224, 3)       0         
_________________________________________________________________
random_flip_1 (RandomFlip)   (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)       

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"])

In [18]:
batch_size = 64
epochs = 20
patience = 2
model.fit(x=x_train,y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), 
          callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience)])

Epoch 1/20
55/55 [==============================] - 44s 793ms/step - loss: 1.6094 - accuracy: 0.1752 - val_loss: 1.6094 - val_accuracy: 0.1828
Epoch 2/20
55/55 [==============================] - 44s 794ms/step - loss: 1.6094 - accuracy: 0.1833 - val_loss: 1.6094 - val_accuracy: 0.1828
Epoch 3/20
51/55 [==========================>...] - ETA: 2s - loss: 1.6094 - accuracy: 0.1881

KeyboardInterrupt: ignored

In [ ]:
# zapisywanie modelu do pliku na colabie
model.save('outdated_network.h5')
# zapisywanie modelu na komputer lokalnie
from google.colab import files
files.download('outdated_network.h5')

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_test.shape

In [ ]:
y_pred.shape

In [ ]:
import sklearn
sklearn.metrics.confusion_matrix(y_test, y_pred.argmax(1))

In [ ]:
from google.colab import files
# uploaded = files.upload()
!wget https://bloomsbythebox.sirv.com/img/product/xlarge/09035D__Rose_Jessica_Pink_50cm.jpg
img = cv2.imread('09035D__Rose_Jessica_Pink_50cm.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img.shape

In [ ]:

img = cv2.resize(img, (224,224))
img = np.array(img, dtype=np.float32)

In [ ]:
img.shape

In [ ]:
img2 = tf.expand_dims(img, axis=0)

In [ ]:
img2.shape

In [ ]:
type(img)

In [ ]:
pred = model.predict(img2) #/255)

In [ ]:
pred

In [ ]:
categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip'] 
for c in categories:
  plt.imshow(img) #.astype('uint8'))
  while pred == c:
    pred_str =+ categories[c] 
    plt.title(f'Prediction: {pred_str}')
    plt.show()